In [1]:
%load_ext autoreload
%autoreload 2

import evolution
from config import Config
from pymoo.algorithms.moo.nsga2 import NSGA2
import os


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

config = Config()
problem = evolution.EVProblem(config)
algorithm = NSGA2(pop_size=config.pop_size,
                    sampling=evolution.SamplingAll(),
                    mutation=evolution.MutationAll(),
                    eliminate_duplicates=True)


In [2]:
from pymoo.optimize import minimize

res = minimize(problem, 
                algorithm,
                callback=evolution.do_every_generations, 
                termination=config.termination)

Blueprint: [0 0 0 1 1 1]
Blueprint: [0 0 0 0 1 0]
Blueprint: [1 0 1 1 1 0]
Blueprint: [1 1 0 0 1 0]
Timestats: blueprint_time: 0.1. fit_time: 38.89
Generation = 1
population error: best = 0.090, mean = 0.489, median = 0.490, worst = 0.887
Best genome: [array([0]), array([0, 0])]
Result = 0.910, params number = 6.73e-06


Blueprint: [0 0 0 1 0 0]
Blueprint: [0 0 0 0 0 0]
Blueprint: [1 0 0 1 1 1]
Blueprint: [1 0 1 0 1 0]
Timestats: blueprint_time: 0.12. fit_time: 41.07
Generation = 2
population error: best = 0.090, mean = 0.100, median = 0.097, worst = 0.117
Best genome: [array([0]), array([0, 0])]
Result = 0.910, params number = 6.73e-06


Blueprint: [1 0 0 0 0 0]


/home/milu/anaconda3/envs/tf-2/lib/python3.9/site-packages/pymoo/algorithms/moo/nsga2.py:191: RuntimeWarning: invalid value encountered in subtract
  dist = np.row_stack([_F, np.full(n_obj, np.inf)]) - np.row_stack([np.full(n_obj, -np.inf), _F])
/home/milu/anaconda3/envs/tf-2/lib/python3.9/site-packages/pymoo/algorithms/moo/nsga2.py:194: RuntimeWarning: invalid value encountered in subtract
  norm = np.max(_F, axis=0) - np.min(_F, axis=0)


Blueprint: [1 1 0 0 1 0]
Blueprint: [1 1 0 1 1 1]
Blueprint: [1 0 0 1 1 1]
Timestats: blueprint_time: 0.13. fit_time: 44.17
Generation = 3
population error: best = 0.090, mean = 0.297, median = 0.106, worst = 0.887
Best genome: [array([0]), array([0, 0])]
Result = 0.910, params number = 6.73e-06


Blueprint: [0 1 0 0 0 0]
Blueprint: [0 1 0 0 1 0]
Blueprint: [1 0 0 1 0 0]
Blueprint: [0 0 0 1 0 0]
Timestats: blueprint_time: 0.12. fit_time: 43.29
Generation = 4
population error: best = 0.090, mean = 0.101, median = 0.098, worst = 0.117
Best genome: [array([0]), array([0, 0])]
Result = 0.910, params number = 6.73e-06


Blueprint: [0 0 0 1 1 0]
Blueprint: [0 1 0 1 0 0]
Blueprint: [0 0 0 1 0 1]
Blueprint: [1 0 0 0 1 0]
Timestats: blueprint_time: 0.11. fit_time: 40.75
Generation = 5
population error: best = 0.090, mean = 0.100, median = 0.096, worst = 0.117
Best genome: [array([0]), array([0, 0])]
Result = 0.910, params number = 6.73e-06


